# Resume Tailor Agent

An intelligent agent that tailors your LaTeX resume to specific job postings while preserving formatting and maintaining accuracy.

## Features

- **LaTeX-Safe**: Preserves LaTeX formatting and syntax
- **Iterative**: Supports multiple revision rounds
- **Job-Focused**: Analyzes job postings and matches requirements
- **ATS-Optimized**: Uses keywords naturally for applicant tracking systems
- **Validation**: Checks LaTeX syntax before output

---

## Setup

Import required libraries and configure the environment.

## API Provider Configuration

This notebook supports multiple AI providers. Configure your credentials in the `.env` file:

### Option 1: OpenAI (Recommended for getting started)
```bash
OPENAI_API_KEY=sk-your-openai-key-here
```

### Option 2: AWS Bedrock (Production-ready)
```bash
# Using long-term API key (recommended)
AWS_BEARER_TOKEN_BEDROCK=your-long-term-bedrock-key
AWS_REGION=us-east-1

# OR using standard AWS credentials
AWS_ACCESS_KEY_ID=your-access-key
AWS_SECRET_ACCESS_KEY=your-secret-key
AWS_REGION=us-east-1
```

The notebook will automatically detect which credentials are available and use them.

---

In [1]:
# Core imports
import os
from pathlib import Path
from dotenv import load_dotenv

# Strands SDK
from strands import Agent, tool

# Utilities
import json
from datetime import datetime

# Load environment variables
load_dotenv()

print("✅ Imports successful!")
print(f"Python Path: {Path.cwd()}")

✅ Imports successful!
Python Path: d:\Strands-agent


## Logging & Observability

Configure logging to trace all agent operations and tool calls.

In [2]:
import logging
import json
from datetime import datetime as dt

# Custom JSON formatter for structured logs
class JsonFormatter(logging.Formatter):
    def format(self, record):
        log_data = {
            "timestamp": self.formatTime(record),
            "level": record.levelname,
            "name": record.name,
            "message": record.getMessage(),
            "function": record.funcName,
            "line": record.lineno
        }
        
        # Add exception info if present
        if record.exc_info:
            log_data["exception"] = self.formatException(record.exc_info)
        
        return json.dumps(log_data)

# Create logs directory
PROJECT_ROOT = Path.cwd()
LOGS_DIR = PROJECT_ROOT / "logs"
LOGS_DIR.mkdir(exist_ok=True)

# Generate log filename with timestamp
log_filename = LOGS_DIR / f"strands_agent_{dt.now().strftime('%Y%m%d_%H%M%S')}.log"

# Configure file handler with JSON formatting
file_handler = logging.FileHandler(log_filename)
file_handler.setFormatter(JsonFormatter())
file_handler.setLevel(logging.DEBUG)

# Configure console handler with simple formatting (for notebook output)
console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter(
    '%(levelname)s | %(name)s | %(message)s'
))
console_handler.setLevel(logging.WARNING)  # Only show warnings/errors in notebook

# Configure the strands logger
strands_logger = logging.getLogger("strands")
strands_logger.setLevel(logging.DEBUG)
strands_logger.addHandler(file_handler)
strands_logger.addHandler(console_handler)

# Prevent duplicate logs
strands_logger.propagate = False

print("✅ Logging configured!")
print(f"   Log file: {log_filename}")
print(f"   Console level: WARNING (errors only)")
print(f"   File level: DEBUG (all traces)")
print()
print("📊 Log includes:")
print("  • All agent operations")
print("  • Tool calls and responses")
print("  • Model interactions")
print("  • Validation results")
print("  • Error traces")

✅ Logging configured!
   Log file: d:\Strands-agent\logs\strands_agent_20251116_113948.log
   Console level: WARNING (errors only)
   File level: DEBUG (all traces)

📊 Log includes:
  • All agent operations
  • Tool calls and responses
  • Model interactions
  • Validation results
  • Error traces


In [3]:
LOGS_DIR

WindowsPath('d:/Strands-agent/logs')

In [4]:
# Helper functions to view and analyze logs

def view_latest_logs(num_lines=50, level_filter=None):
    """
    View the latest log entries from the current log file.
    
    Args:
        num_lines: Number of recent log lines to display
        level_filter: Filter by log level (e.g., 'ERROR', 'DEBUG', 'WARNING')
    """
    try:
        with open(log_filename, 'r') as f:
            lines = f.readlines()
        
        # Filter by level if specified
        if level_filter:
            filtered_lines = []
            for line in lines:
                try:
                    log_entry = json.loads(line)
                    if log_entry.get('level') == level_filter:
                        filtered_lines.append(line)
                except:
                    continue
            lines = filtered_lines
        
        # Show last N lines
        recent_lines = lines[-num_lines:]
        
        print(f"📋 Showing last {len(recent_lines)} log entries:")
        print(f"   Filter: {level_filter if level_filter else 'All levels'}")
        print(f"   Total entries: {len(lines)}")
        print("=" * 80)
        
        for line in recent_lines:
            try:
                log_entry = json.loads(line)
                print(f"{log_entry['timestamp']} | {log_entry['level']:8} | {log_entry['name']}")
                print(f"  → {log_entry['message']}")
                if 'exception' in log_entry:
                    print(f"  ⚠️  {log_entry['exception']}")
                print()
            except:
                print(line.strip())
    
    except FileNotFoundError:
        print(f"❌ Log file not found: {log_filename}")
    except Exception as e:
        print(f"❌ Error reading logs: {e}")


def count_tool_calls():
    """Count how many times each tool was called."""
    try:
        with open(log_filename, 'r') as f:
            lines = f.readlines()
        
        tool_calls = {}
        for line in lines:
            try:
                log_entry = json.loads(line)
                message = log_entry.get('message', '')
                
                # Look for tool call patterns
                if 'tool' in message.lower() and 'call' in message.lower():
                    # Extract tool name (you may need to adjust this based on actual log format)
                    if 'merge_sections' in message:
                        tool_calls['merge_sections'] = tool_calls.get('merge_sections', 0) + 1
                    elif 'read_file' in message:
                        tool_calls['read_file'] = tool_calls.get('read_file', 0) + 1
                    elif 'validate_latex' in message:
                        tool_calls['validate_latex'] = tool_calls.get('validate_latex', 0) + 1
                    elif 'extract_section' in message:
                        tool_calls['extract_section'] = tool_calls.get('extract_section', 0) + 1
            except:
                continue
        
        print("🔧 Tool Call Summary:")
        print("=" * 40)
        for tool, count in sorted(tool_calls.items(), key=lambda x: x[1], reverse=True):
            print(f"  {tool:20} : {count:3} calls")
        
        if not tool_calls:
            print("  No tool calls detected in logs")
    
    except Exception as e:
        print(f"❌ Error analyzing tool calls: {e}")


def export_logs_to_readable(output_file=None):
    """Export JSON logs to a human-readable format."""
    if output_file is None:
        output_file = LOGS_DIR / f"readable_log_{dt.now().strftime('%Y%m%d_%H%M%S')}.txt"
    
    try:
        with open(log_filename, 'r') as f:
            lines = f.readlines()
        
        with open(output_file, 'w') as out:
            out.write(f"Strands Agent Log - Readable Format\n")
            out.write(f"Generated: {dt.now()}\n")
            out.write(f"Source: {log_filename}\n")
            out.write("=" * 80 + "\n\n")
            
            for line in lines:
                try:
                    log_entry = json.loads(line)
                    out.write(f"[{log_entry['timestamp']}] {log_entry['level']}\n")
                    out.write(f"Module: {log_entry['name']}\n")
                    out.write(f"Message: {log_entry['message']}\n")
                    if 'exception' in log_entry:
                        out.write(f"Exception:\n{log_entry['exception']}\n")
                    out.write("-" * 80 + "\n\n")
                except:
                    out.write(line)
        
        print(f"✅ Readable log exported to: {output_file}")
        return output_file
    
    except Exception as e:
        print(f"❌ Error exporting logs: {e}")


print("✅ Log analysis functions defined:")
print("  - view_latest_logs(num_lines=50, level_filter=None)")
print("  - count_tool_calls()")
print("  - export_logs_to_readable(output_file=None)")
print()
print("Examples:")
print('  view_latest_logs(20)                    # Last 20 log entries')
print('  view_latest_logs(level_filter="ERROR")  # Only errors')
print('  count_tool_calls()                      # Tool usage stats')
print('  export_logs_to_readable()               # Export to .txt file')

✅ Log analysis functions defined:
  - view_latest_logs(num_lines=50, level_filter=None)
  - count_tool_calls()
  - export_logs_to_readable(output_file=None)

Examples:
  view_latest_logs(20)                    # Last 20 log entries
  view_latest_logs(level_filter="ERROR")  # Only errors
  count_tool_calls()                      # Tool usage stats
  export_logs_to_readable()               # Export to .txt file


## Configuration

Set up paths and verify environment.

In [5]:
# Project paths
PROJECT_ROOT = Path.cwd()
PROMPTS_DIR = PROJECT_ROOT / "prompts"
DATA_DIR = PROJECT_ROOT / "data"
ORIGINAL_RESUME_DIR = DATA_DIR / "original"
JOB_POSTINGS_DIR = DATA_DIR / "job_postings"
OUTPUT_DIR = DATA_DIR / "tailored_versions"

# Detect which API credentials are available
print("🔍 Checking API credentials...")
print()

has_openai = bool(os.getenv('OPENAI_API_KEY'))
has_bedrock_token = bool(os.getenv('AWS_BEARER_TOKEN_BEDROCK'))
has_aws_creds = bool(os.getenv('AWS_ACCESS_KEY_ID'))

from strands.models import openai

if has_openai:
    print("✅ OpenAI API key found")
    #MODEL_ID = openai.OpenAIModel(model_id="gpt-5-mini")
    MODEL_ID = openai.OpenAIModel(
    model_id="gpt-5-mini",  # Note: prompt caching works best with gpt-4o and newer models
    params={
        "store": True,  # Enable prompt caching
        "metadata": {
            "purpose": "resume_tailoring"  # Optional: track usage
        }
    }
)
elif has_bedrock_token:
    print("✅ AWS Bedrock bearer token found")
    MODEL_PROVIDER = "bedrock"
    MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
elif has_aws_creds:
    print("✅ AWS credentials found")
    MODEL_PROVIDER = "bedrock"
    MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
else:
    print("⚠️  Warning: No API credentials found!")
    print("Please set one of the following in .env file:")
    print("  - OPENAI_API_KEY (for OpenAI)")
    print("  - AWS_BEARER_TOKEN_BEDROCK (for Bedrock)")
    print("  - AWS_ACCESS_KEY_ID + AWS_SECRET_ACCESS_KEY (for AWS)")
    MODEL_PROVIDER = None
    MODEL_ID = None

print()
print(f"🤖 Selected Model: {MODEL_ID}")

# Verify directories exist
print()
print(f"📁 Project directories:")
print(f"  Prompts: {PROMPTS_DIR.exists()} - {PROMPTS_DIR}")
print(f"  Data: {DATA_DIR.exists()} - {DATA_DIR}")
print(f"  Output: {OUTPUT_DIR.exists()} - {OUTPUT_DIR}")

🔍 Checking API credentials...

✅ OpenAI API key found

🤖 Selected Model: <strands.models.openai.OpenAIModel object at 0x0000010D37980050>

📁 Project directories:
  Prompts: True - d:\Strands-agent\prompts
  Data: True - d:\Strands-agent\data
  Output: True - d:\Strands-agent\data\tailored_versions


## Load System Prompts

Load agent instructions from separate files for easy iteration.

In [6]:
def load_prompt(filename: str) -> str:
    """Load a prompt from the prompts directory."""
    prompt_path = PROMPTS_DIR / filename
    if not prompt_path.exists():
        print(f"⚠️  Warning: {filename} not found. Using default prompt.")
        return ""
    
    with open(prompt_path, 'r', encoding='utf-8') as f:
        content = f.read()
    print(f"✅ Loaded {filename} ({len(content)} chars)")
    return content

# Load prompts
system_prompt = load_prompt("system_prompt.txt")
latex_rules = ""

# Combine prompts
full_prompt = f"{system_prompt}\n\n{latex_rules}".strip()

print(f"\n📝 Full system prompt: {len(full_prompt)} characters")

✅ Loaded system_prompt.txt (7417 chars)

📝 Full system prompt: 7416 characters


## Custom Tools for Resume Tailoring

Define specialized tools for LaTeX resume processing.

In [7]:
@tool
def read_file(filepath: str) -> str:
    """
    Read a file and return its contents.
    
    Args:
        filepath: Path to the file (relative to project root or absolute)
    
    Returns:
        The file contents as a string
    """
    try:
        path = Path(filepath)
        if not path.is_absolute():
            path = PROJECT_ROOT / filepath
        
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        return f"Error: File not found at {filepath}"
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def write_file(filepath: str, content: str) -> str:
    """
    Write content to a file.
    
    Args:
        filepath: Path to the file (relative to project root or absolute)
        content: Content to write
    
    Returns:
        Success message with file path
    """
    try:
        path = Path(filepath)
        if not path.is_absolute():
            path = PROJECT_ROOT / filepath
        
        # Create parent directories if needed
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        
        return f"Successfully wrote {len(content)} characters to {path}"
    except Exception as e:
        return f"Error writing file: {str(e)}"


@tool
def validate_latex(latex_content: str) -> dict:
    """
    Validate LaTeX syntax by checking for common issues.
    
    Args:
        latex_content: The LaTeX content to validate
    
    Returns:
        Dictionary with validation results (is_valid, errors, warnings)
    """
    errors = []
    warnings = []
    
    # Check for balanced braces
    if latex_content.count('{') != latex_content.count('}'):
        errors.append("Unbalanced curly braces { }")
    
    # Check for balanced brackets
    if latex_content.count('[') != latex_content.count(']'):
        errors.append("Unbalanced square brackets [ ]")
    
    # Check for document structure
    if '\\documentclass' not in latex_content:
        warnings.append("No \\documentclass found")
    
    if '\\begin{document}' not in latex_content:
        errors.append("Missing \\begin{document}")
    
    if '\\end{document}' not in latex_content:
        errors.append("Missing \\end{document}")
    
    # Check for common LaTeX commands
    lines = latex_content.split('\n')
    for i, line in enumerate(lines, 1):
        # Check for unescaped special characters in regular text
        if '%' in line and '\\%' not in line:
            # This might be a comment, so it's just a warning
            pass
    
    is_valid = len(errors) == 0
    
    return {
        "is_valid": is_valid,
        "errors": errors,
        "warnings": warnings,
        "summary": f"{'✅ Valid' if is_valid else '❌ Invalid'} LaTeX ({len(errors)} errors, {len(warnings)} warnings)"
    }


@tool
def extract_keywords(text: str) -> list:
    """
    Extract important keywords from text (job posting or resume section).
    
    Args:
        text: Text to extract keywords from
    
    Returns:
        List of keywords (skills, technologies, requirements)
    """
    # Common technical keywords and skills
    import re
    
    # Simple keyword extraction (can be enhanced with NLP)
    keywords = set()
    
    # Common technical skills patterns
    patterns = [
        r'\b(Python|Java|JavaScript|TypeScript|C\+\+|Ruby|Go|Rust|Swift)\b',
        r'\b(AWS|Azure|GCP|Docker|Kubernetes|Jenkins)\b',
        r'\b(React|Angular|Vue|Node\.js|Django|Flask|Spring)\b',
        r'\b(SQL|PostgreSQL|MySQL|MongoDB|Redis)\b',
        r'\b(Git|CI/CD|Agile|Scrum|DevOps|REST|API)\b',
        r'\b(Machine Learning|AI|Data Science|Analytics)\b',
    ]
    
    for pattern in patterns:
        matches = re.finditer(pattern, text, re.IGNORECASE)
        for match in matches:
            keywords.add(match.group(1))
    
    return sorted(list(keywords))


print("✅ Custom tools defined:")
print("  - read_file()")
print("  - write_file()")
print("  - validate_latex()")
print("  - extract_keywords()")

✅ Custom tools defined:
  - read_file()
  - write_file()
  - validate_latex()
  - extract_keywords()


## Create the Resume Tailor Agent

Initialize the agent with system prompts and custom tools.

In [8]:
# Create agent with automatic provider detection
MODEL_PROVIDER = "OpenAI"
if MODEL_PROVIDER is None:
    print("❌ Cannot create agent: No API credentials found")
    print("Please configure API credentials in .env file")
else:
    # Import section updater tools
    from tools.section_updater import (
        extract_section,
        replace_section,
        update_subtitle,
        merge_sections,
        get_section_names
    )
    
    # Create agent with detected provider and ALL tools
    resume_agent = Agent(
        model=MODEL_ID,
        system_prompt=full_prompt if full_prompt else "You are a helpful resume tailoring assistant.",
        tools=[
            read_file,
            write_file,
            validate_latex,
            extract_keywords,
            # Section updater tools
            extract_section,
            replace_section,
            update_subtitle,
            merge_sections,
            get_section_names
        ]
    )

    print("✅ Resume Tailor Agent created!")
    print(f"   Provider: {MODEL_PROVIDER}")
    print(f"   Model: {MODEL_ID}")
    print(f"   Tools: {len(resume_agent.tool_names)} tools available")
    print(f"   System prompt: {len(full_prompt) if full_prompt else 0} characters")
    print()
    print("Available tools:")
    print("  • read_file, write_file, validate_latex, extract_keywords")
    print("  • extract_section, replace_section, update_subtitle")
    print("  • merge_sections, get_section_names")
    print()
    print("💡 Tip: You can change the model by editing MODEL_PROVIDER and MODEL_ID in the configuration cell above")

✅ Resume Tailor Agent created!
   Provider: OpenAI
   Model: <strands.models.openai.OpenAIModel object at 0x0000010D37980050>
   Tools: 9 tools available
   System prompt: 7416 characters

Available tools:
  • read_file, write_file, validate_latex, extract_keywords
  • extract_section, replace_section, update_subtitle
  • merge_sections, get_section_names

💡 Tip: You can change the model by editing MODEL_PROVIDER and MODEL_ID in the configuration cell above


---

## Usage Examples

Below are examples of how to use the resume tailor agent.

### Example 3: Resume Tailoring Workflow

**Note**: You'll need to place your actual LaTeX resume in `data/original/resume.tex`

In [ ]:
# Define file paths 
original_resume = "data/original/Data_engineer.tex.tex"      # Your resume here
job_posting = "data/job_postings/quanlom.txt"         # Job posting
output_file = "data/tailored_versions/new_mini_resume.tex"  # Output

# Instructions for the agent
tailoring_request = f"""
You are now in ANALYSIS MODE.

First, you MUST load the inputs using your tools:

1. Call the `read_file` tool with the path "{original_resume}" to load my LaTeX resume.
2. Call the `read_file` tool with the path "{job_posting}" to load the job posting text.

After you have loaded BOTH files, do the following:

1. Analyze the job posting and extract key requirements, responsibilities, and skills.
2. Compare them with the content of the LaTeX resume.
3. Suggest specific, concrete edits to the resume, for example:
   - Which bullets to rephrase (and provide the new wording).
   - Which skills to emphasize or de-emphasize.
   - Any reordering of sections or bullets that would improve alignment.
4. Point out any major gaps where the job requires something NOT present in the resume.

IMPORTANT:
- Do NOT output LaTeX in this step—only natural-language analysis and suggestions.
- Do NOT assume the resume or job posting are already in the conversation; you must read them using `read_file` first.
"""

# Get initial analysis
print("Analyzing resume and job posting...\n")
analysis = resume_agent(tailoring_request)
print(analysis)

Analyzing resume and job posting...


Tool #1: read_file

Tool #2: read_file
I have loaded both files and reviewed the job posting and your LaTeX resume. Below is a focused analysis, mapping of requirements → resume content, concrete edit recommendations (including exact rephrases you can drop into the LaTeX), and identified gaps.

1) Job posting — key requirements & keywords
- Must-have / high-priority:
  - 100% coding in Python; proven ability to build frameworks, APIs, or developer platforms
  - Hands-on experience building AI solutions + supporting infrastructure
  - Multi-agent workflows and orchestration frameworks
  - RAG patterns and retrieval pipelines
  - Docker, containers, Kubernetes; deploying containers and managing enterprise-grade K8s
  - LLM orchestration frameworks, multi-agent architectures, conversational AI pipelines
  - Emphasis on custom development (developer-facing code) over purely infra setup

- Nice-to-have / specific product names:
  - MCP-Context-Forge (ag

### Example 4: Iterative Refinement

After getting suggestions, you can iterate:

In [10]:
# # Continue the conversation for refinement
# refinement_request = """
# Based on your analysis, please:
# 1. Focus on highlighting my AWS and Python experience
# 2. Emphasize any machine learning projects
# 3. Ensure keywords match the job posting for ATS
# 4. Keep the resume to 1 page if possible

# Show me the specific sections that should change.
# """

# refinement = resume_agent(refinement_request)
# print(refinement)

### Example 5: Generate Final Tailored Resume

In [11]:
# Generate the final version
final_request = f"""
You are now in GENERATE MODE.

Your goal is to produce a tailored LaTeX resume for the job posting we analyzed earlier and save it to "{output_file}".

You have these tools: read_file, write_file, validate_latex, extract_section, replace_section, update_subtitle, merge_sections, get_section_names.

Follow these steps:

1. Call `read_file` with "{original_resume}" to load the ORIGINAL LaTeX resume.
2. Identify the sections that should be updated for this role
   (typically "Professional Summary", "Professional Experience", and "Technical Proficiencies").
   You may use `get_section_names` and `extract_section` to inspect them.
3. For each relevant section, rewrite the LaTeX content using the SAME macros and structure
   (e.g., \\resumeEntryStart, \\resumeItem, \\resumeEntryS, etc.) so that it better matches
   the job posting requirements, based on your previous analysis.
4. Use `merge_sections` to merge your updated section LaTeX into the original file:
   - original_file = "{original_resume}"
   - updated_sections = {{
       "Professional Summary": <your new LaTeX for that section>,
       "Professional Experience": <your new LaTeX for that section>,
       "Technical Proficiencies": <your new LaTeX for that section>,
       "subtitle": <optional new subtitle/job title>
     }}
   - output_file = "{output_file}"


FINAL RESPONSE REQUIREMENTS:
- In your final assistant message, RETURN ONLY the COMPLETE UPDATED LATEX SOURCE
  (the contents of "{output_file}") with NO commentary, NO markdown fences, and NO tool logs.
- Do NOT modify the LaTeX preamble (\\documentclass, \\usepackage, \\def, \\newcommand).
- Only modify text inside existing commands and environments (sections, bullets, skills, etc.).
"""

# This is burning tokens, shouldn't let the agent do this.
# "5. After merging, call `read_file` on "{output_file}" to get the full updated LaTeX content.
# 6. Call `validate_latex` on the full updated LaTeX to check for obvious syntax issues
#    (unbalanced braces, missing \\begin{{document}} / \\end{{document}}, etc.).
#    If validation reports errors, fix the LaTeX and re-run `validate_latex` until it passes."

result = resume_agent(final_request)
print(result)


Tool #3: read_file

Tool #4: get_section_names

Tool #5: extract_section

Tool #6: replace_section

Tool #7: replace_section

Tool #8: merge_sections

Tool #9: update_subtitle

Tool #10: write_file

Tool #11: read_file

Tool #12: validate_latex

Tool #13: read_file

Tool #14: validate_latex
%-------------------------------------
% LaTeX Resume for Software Engineers
% Author : Leslie Cheng
% License : MIT
%-------------------------------------

\documentclass[a4paper,10.5pt]{article}[leftmargin=*]

\usepackage[empty]{fullpage}
\usepackage{enumitem}
\usepackage{ifxetex}
\ifxetex
  \usepackage{fontspec}
  \usepackage[xetex]{hyperref}
\else
  \usepackage[utf8]{inputenc}
  \usepackage[T1]{fontenc}
  \usepackage[pdftex]{hyperref}
\fi
\usepackage{fontawesome}
\usepackage[sfdefault,light]{FiraSans}
\usepackage{anyfontsize}
\usepackage{xcolor}
\usepackage{tabularx}

%-------------------------------------------------- SETTINGS HERE --------------------------------------------------
% Header se

### Example 6: Validate Output

In [12]:
# Direct tool invocation for validation
if Path(output_file).exists():
    with open(output_file, 'r', encoding='utf-8') as f:
        tailored_content = f.read()
    
    # Validate using tool directly
    validation = resume_agent.tool.validate_latex(latex_content=tailored_content)
    
    # Handle different return formats (dict, string, or other)
    import json
    if isinstance(validation, dict):
        validation_dict = validation
    elif isinstance(validation, str):
        # Try to parse as JSON if it's a string
        try:
            validation_dict = json.loads(validation)
        except json.JSONDecodeError:
            # If not JSON, print the raw result
            print("Validation Results (raw):")
            print(validation)
            validation_dict = None
    else:
        print(f"Validation returned unexpected type: {type(validation)}")
        print(f"Value: {validation}")
        validation_dict = None
    
    if validation_dict:
        print("Validation Results:")
        print(f"  Valid: {validation_dict.get('is_valid', 'N/A')}")
        print(f"  Errors: {len(validation_dict.get('errors', []))}")
        print(f"  Warnings: {len(validation_dict.get('warnings', []))}")
        
        if validation_dict.get('errors'):
            print("\nErrors found:")
            for error in validation_dict['errors']:
                print(f"  - {error}")
        
        if validation_dict.get('warnings'):
            print("\nWarnings:")
            for warning in validation_dict['warnings']:
                print(f"  - {warning}")
        
        if validation_dict.get('summary'):
            print(f"\n{validation_dict['summary']}")
else:
    print(f"File not found: {output_file}")

Validation Results:
  Valid: N/A
  Errors: 0
  Warnings: 0


---

## Helper Functions

Utility functions for common tasks.

In [13]:
# def quick_tailor(resume_path: str, job_path: str, output_path: str, instructions: str = ""):
#     """
#     Quick one-shot resume tailoring.
    
#     Args:
#         resume_path: Path to original resume
#         job_path: Path to job posting
#         output_path: Path for tailored resume
#         instructions: Additional instructions for the agent
#     """
#     prompt = f"""
# Tailor my resume for this job posting.

# Resume: {resume_path}
# Job Posting: {job_path}
# Output: {output_path}

# Steps:
# 1. Read both files
# 2. Analyze job requirements
# 3. Tailor resume content (preserve LaTeX formatting)
# 4. Validate LaTeX syntax
# 5. Save to output path

# {instructions if instructions else ''}
# """
    
#     response = resume_agent(prompt)
#     return response


# def batch_tailor(resume_path: str, job_folder: str, output_folder: str):
#     """
#     Tailor resume for multiple job postings.
    
#     Args:
#         resume_path: Path to original resume
#         job_folder: Folder containing job posting files
#         output_folder: Folder for tailored resumes
#     """
#     job_dir = Path(job_folder)
#     output_dir = Path(output_folder)
#     output_dir.mkdir(parents=True, exist_ok=True)
    
#     results = []
    
#     for job_file in job_dir.glob("*.txt"):
#         output_name = f"resume_{job_file.stem}.tex"
#         output_path = output_dir / output_name
        
#         print(f"\nTailoring for: {job_file.name}")
#         result = quick_tailor(resume_path, str(job_file), str(output_path))
#         results.append({"job": job_file.name, "output": output_name, "result": result})
    
#     return results


# print("✅ Helper functions defined:")
# print("  - quick_tailor()")
# print("  - batch_tailor()")

---

## Next Steps

1. **Add your resume**: Place your LaTeX resume in `data/original/resume.tex`
2. **Add job postings**: Save job postings as `.txt` files in `data/job_postings/`
3. **Run tailoring**: Use the examples above to tailor your resume
4. **Iterate**: Work with the agent to refine the output
5. **Validate**: Check LaTeX syntax before compiling
6. **Compile**: Use `pdflatex` or your LaTeX editor to generate PDF

### Tips for Best Results

- Start with analysis and suggestions before generating the full resume
- Be specific about what aspects to highlight
- Review the agent's suggestions before applying them
- Always validate LaTeX syntax
- Keep conversation context for iterative improvements
- Save different versions for different job types

### Troubleshooting

- **LaTeX errors**: Use `validate_latex()` tool to check syntax
- **Agent not following instructions**: Refine the system prompt in `prompts/system_prompt.txt`
- **Missing features**: Add custom tools as needed
- **Context lost**: Use conversation memory or save intermediate results